In [1]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [4]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = np.load('communityAggregatedMatrix15.npy')
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [5]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [6]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix,dates = getMatrixData()
matrix = matrix.astype(float)
matrix.shape

Raw shape:  (7374816, 5)
Days:  638
Matrix Shape:  (638, 256)


(638, 256)

In [7]:
min(dates), max(dates)

(numpy.datetime64('2017-01-01T00:00:00.000000000'),
 numpy.datetime64('2018-09-30T00:00:00.000000000'))

In [41]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [42]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [43]:
# stations = list(set(dataRaw.start_id))
# n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [44]:
#dates = list(dataOut.index)
DOW = list(pd.to_datetime(dates).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [45]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(input_dim, hiddenDim[0])
        self.linear2 = nn.Linear(hiddenDim[0], hiddenDim[1])
        self.linear3 = nn.Linear(hiddenDim[1], hiddenDim[2])
        self.linear4 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.linear3)
        self.init_weights(self.linear4)
        
    def forward(self, x):
        #x, edge_index = data.x, data.edge_index
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear4(x)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x
            

In [46]:
matrixtensor = torch.tensor(matrix).float()

In [47]:
input_dim = matrix.shape[1]

In [48]:
hiddenDim = [500,100,20]
n_classes = 2

learning_rate=0.001
batch_size = 50
num_epochs = 200

In [49]:
Y = torch.tensor(np.array(DOW), dtype=torch.long)

In [50]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, Y) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [52]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, y = data
        X = X.cuda()
        y = y.cuda()
#         print(X.shape)
#         print(y.shape)
        # ===================forward=====================
        output = model(X)
        #print(output.shape)
        loss = criterion(output, y)
        #MSE_loss = nn.MSELoss()(output, y)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item()))

epoch [1/200], loss:0.6540
epoch [2/200], loss:0.6220
epoch [3/200], loss:0.5688
epoch [4/200], loss:0.4982
epoch [5/200], loss:0.5990
epoch [6/200], loss:0.4122
epoch [7/200], loss:0.5436
epoch [8/200], loss:0.4496
epoch [9/200], loss:0.4203
epoch [10/200], loss:0.4297
epoch [11/200], loss:0.3695
epoch [12/200], loss:0.2858
epoch [13/200], loss:0.3634
epoch [14/200], loss:0.2495
epoch [15/200], loss:0.2886
epoch [16/200], loss:0.2954
epoch [17/200], loss:0.3133
epoch [18/200], loss:0.1661
epoch [19/200], loss:0.2900
epoch [20/200], loss:0.2162
epoch [21/200], loss:0.1572
epoch [22/200], loss:0.1134
epoch [23/200], loss:0.0982
epoch [24/200], loss:0.1350
epoch [25/200], loss:0.2521
epoch [26/200], loss:0.1705
epoch [27/200], loss:0.2015
epoch [28/200], loss:0.1180
epoch [29/200], loss:0.1604
epoch [30/200], loss:0.3206
epoch [31/200], loss:0.0929
epoch [32/200], loss:0.1331
epoch [33/200], loss:0.1505
epoch [34/200], loss:0.1184
epoch [35/200], loss:0.1448
epoch [36/200], loss:0.1128
e

In [53]:
model.eval()
with torch.no_grad():
    y = np.array(DOW)
    predList = model(matrixtensor.cuda()).cpu().numpy()
    pred = np.argmax(predList,axis=1)    
    print('Accuracy: {:.4f}'.format( np.sum(pred == y) / pred.shape[0] ))

Accuracy: 0.9765


In [54]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(638, 20)

In [8]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [9]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [11]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date
len(holidayDates)

30

In [58]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = pd.to_datetime(dates)
dates = [str(d.date()) for d in dates]

In [59]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [60]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [61]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [62]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [63]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Thresh

In [64]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.285714,0.001642,0.833333,0.172414,0.01
1,Global,0.666667,0.004926,0.842105,0.551724,0.03
2,Global,0.777778,0.006568,0.840000,0.724138,0.04
3,Global,0.766667,0.013136,0.741935,0.793103,0.05
4,Global,0.716418,0.022989,0.631579,0.827586,0.06
5,Global,0.657534,0.032841,0.545455,0.827586,0.07
6,Global,0.600000,0.044335,0.470588,0.827586,0.08
7,Global,0.586957,0.059113,0.428571,0.931034,0.10
8,Global,0.346154,0.164204,0.212598,0.931034,0.20
9,Global,0.254545,0.267652,0.146597,0.965517,0.30


In [69]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(matrix)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [70]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(638, 20)


In [71]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergenc

In [72]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.285714,0.001642,0.833333,0.172414,0.01
1,Global,0.458333,0.013136,0.578947,0.379310,0.03
2,Global,0.444444,0.021346,0.480000,0.413793,0.04
3,Global,0.500000,0.026273,0.483871,0.517241,0.05
4,Global,0.537313,0.032841,0.473684,0.620690,0.06
5,Global,0.547945,0.039409,0.454545,0.689655,0.07
6,Global,0.525000,0.049261,0.411765,0.724138,0.08
7,Global,0.456522,0.068966,0.333333,0.724138,0.10
8,Global,0.371795,0.160920,0.228346,1.000000,0.20
9,Global,0.263636,0.266010,0.151832,1.000000,0.30
